# Reference

## Streaming With Tweepy

http://docs.tweepy.org/en/v3.6.0/streaming_how_to.html

Some features of Tweepy streaming are not covered here. See streaming.py in the Tweepy source code.

### Summary
The Twitter streaming API is used to download twitter messages in real time. It is useful for obtaining a high volume of tweets, or for creating a live feed using a site stream or user stream. See the [Twitter Streaming API Documentation](https://developer.twitter.com/en/docs).

The streaming api is quite different from the REST api because the REST api is used to pull data from twitter but the streaming api pushes messages to a persistent session. This allows the streaming api to download more data in real time than could be done using the REST API.

In Tweepy, an instance of **tweepy.Stream** establishes a streaming session and routes messages to **StreamListener** instance. The **on_data** method of a stream listener receives all messages and calls functions according to the message type. The default **StreamListener** can classify most common twitter messages and routes them to appropriately named methods, but these methods are only stubs.

Therefore using the streaming api has three steps.

1. Create a class inheriting from StreamListener
2. Using that class create a Stream object
3. Connect to the Twitter API using the Stream.

In [1]:
import tweepy

In [2]:
import config

consumer_key = config.twitter_anidata_consumer_key
consumer_secret = config.twitter_anidata_consumer_secret
access_token = config.twitter_anidata_access_token
access_token_secret = config.twitter_anidata_access_token_secret

# create an OAuthHandler instance
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth) 

# Capture the Stream

Note: It appears that `if not hasattr(status, 'retweeted_status'):` is a more accurate way of filtering out retweets than `if status.retweeted == False:`. It would be preferrable to determine this explictly, rather than emperically. Empirically, using the latter seemed to include some suspected retweets in the results. Specifically there was a popular retweeet involving Tom Brady's super bowl history which kept creeping in with the latter, but not the former. 

Also note, with regard to the former, that merely using `if not status.retweeted_status` generates an error when the `retweeted_status` attibute does not exist. 

In [4]:
### Create a StreamListener ###

# override tweepy.StreamListener to add logic to on_status

class MyStreamListener(tweepy.StreamListener):

    def on_status(self, status):
        # https://developer.twitter.com/en/docs/tweets/data-dictionary/overview/tweet-object
        #if status.retweeted == False:
            #print("retweeted = ", status.retweeted, ": ", status.user.screen_name, ": ", status.text)
        
        #https://stackoverflow.com/questions/610883/how-to-know-if-an-object-has-an-attribute-in-python
        if not hasattr(status, 'retweeted_status'):
            print("retweeted = ", status.retweeted, ": ", status.user.screen_name, ": ", status.text)
        
    def on_error(self, status_code):
        if status_code == 420:
            #returning False in on_error disconnects the stream
            return False

        # returning non-False reconnects the stream, with backoff.
        
# Create a Stream object of class MyStreamListener

myStreamListener = MyStreamListener()
myStream = tweepy.Stream(auth = api.auth, listener=myStreamListener, lang='en')

# Start a Stream

myStream.filter(track=['atlanta'])

retweeted =  False :  BGirlVentures :  RT @RaceCummings: #foodie #founder #tech #gamechanger #entrepreneur  #business #qoutes  #blackentrepreneur #Atlanta… https://t.co/evOWpw9Dkn
retweeted =  False :  youngspice8 :  @JeffreeStar Atlanta!!!
retweeted =  False :  iTdm_Ink :  I know the third section 😎
retweeted =  False :  CaseyOliver15 :  @JosephBatten621 @BravesLiz he has the swag that atlanta needs. he would fit in great with our all Hispanic squad we are building. hahaha
retweeted =  False :  willshull :  Stumbled into a New Division show in Atlanta tonight https://t.co/JKFGnx2ato
retweeted =  False :  Sehunsknees :  Alright, its sleepy time, tomorrow is finally the day I get to go to Atlanta. Goodnight and I love you all
retweeted =  False :  shauseman :  @fansince1988 @MikeBellATL @Gritzblitz56 Now that United has established themselves as the most passionate Atlanta… https://t.co/tdUWBpucNu
retweeted =  False :  gerthavillarre9 :  RT: Morning! We will love for your to come join u

KeyboardInterrupt: 

# Store the Data